## Importar librerias

In [17]:
from simpy import Environment, Container, Resource
import random
import pandas as pd

## Variables de configuracion

In [18]:
NUM_VEHICULOS = 5
NUM_ASIENTOS_VEHICULO = 4
DURACION_SIMULACION = 180
# Nombre de la colonia y distancia (km) relativa al punto de taxi
COLONIAS_SETTINGS = [
    { "nombre": "A", "distancia": 1 }
    , { "nombre": "B", "distancia": 3 }
    , { "nombre": "C", "distancia": 2 }
]

## Definicion de clases

### Clases para representar destino de los pasajeros

In [19]:
class Colonia:
    def __init__(self, nombre: str, distancia_absoluta: int):
        self.nombre = nombre
        self.distancia_absoluta = distancia_absoluta
        
class Destino:
    def __init__(self, colonia: Colonia, punto: int):
        self.colonia = colonia
        self.punto = punto
        self.distancia_absoluta = self.colonia.distancia_absoluta * (self.punto / 5)

### Clase para representar el punto de taxi

In [20]:
class PuntoTaxi:
    def __init__(self, env: Environment, colonias: [Colonia], vehiculos):
        self.env = env
        self.vehiculos = Resource(
            env
            , capacity = NUM_VEHICULOS
        )
        # Hay una fila de pasajeros por colonia
        self.colas_pasajeros: dict = { c.nombre: [] for c in colonias }
        self.cola_vehiculos = vehiculos

### Clase para representar pasajeros

In [21]:
class Pasajero:
    def __init__(self, env: Environment, id: int, destino: Destino, punto_taxi: PuntoTaxi):
        self.env = env
        self.id = id
        self.destino = destino
        self.punto_taxi = punto_taxi
        self.env.process(self.start())
        self.vehiculo: int = None
        self.eventos = pd.DataFrame()

    def start(self):
        print(f"{self.env.now}: pasajero con id {self.id} llego al punto con destino: colonia {self.destino.colonia.nombre}, punto {self.destino.punto}")

        self.eventos = pd.concat([
            self.eventos
            , pd.DataFrame({
                "pasajero": [self.id]
                , "evento": ["LLEGO AL PUNTO"]
                , "tiempo": [self.env.now]
            })
        ])

        with self.punto_taxi.vehiculos.request() as req:
            colonia = self.destino.colonia.nombre
            cola_pasajeros = self.punto_taxi.colas_pasajeros[colonia]
            
            # Deben haber minimo 4 pasajeros con el mismo destino para realizar el viaje
            if len(cola_pasajeros) < NUM_ASIENTOS_VEHICULO:
                return
            
            # Obtener primeros 4 pasajeros por orden de llegada
            # Ordenar de forma ascendente en funcion de la distancia absoluta de su destino
            siguientes_pasajeros = sorted([ cola_pasajeros.pop(0) for _ in range(NUM_ASIENTOS_VEHICULO) ], key = lambda p: p.destino.punto)
            # siguientes_pasajeros = [ cola_pasajeros.pop(0) for _ in range(NUM_ASIENTOS_VEHICULO) ]
            ids_pasajeros = [ p.id for p in siguientes_pasajeros ]
            
            print(f"{self.env.now}: pasajeros {ids_pasajeros} con destino a colonia {colonia} listos para subir a taxi")
            
            yield req

            # Obtener el primer taxi de la cola
            vehiculo: Vehiculo = self.punto_taxi.cola_vehiculos.pop(0)

            for pasajero in siguientes_pasajeros:
                pasajero.eventos = pd.concat([
                    pasajero.eventos
                    , pd.DataFrame({
                        "pasajero": [pasajero.id]
                        , "evento": ["SUBIO A VEHICULO"]
                        , "tiempo": [self.env.now]
                    })
                ])

                pasajero.vehiculo = vehiculo.id

            vehiculo.eventos = pd.concat([
                vehiculo.eventos
                , pd.DataFrame({
                    "vehiculo": [vehiculo.id]
                    , "evento": ["HACIA EL DESTINO"]
                    , "tiempo": [self.env.now]
                })
            ])

            print(f"{self.env.now}: pasajeros {ids_pasajeros} subieron a vehiculo con id {vehiculo.id}")

            # self.env.process(vehiculo.start(siguientes_pasajeros))
            
            velocidad_const = random.randint(15, 25)
            
            t_abs = [ int(60 * p.destino.distancia_absoluta / velocidad_const) for p in siguientes_pasajeros ]
            t_relat = [t_abs[0]] + [ t_abs[i] - t_abs[i - 1] for i in range(1, len(t_abs)) ]

            now = self.env.now

            # Pasajeros estan dentro del vehiculo
            for pasajero, tiempo in zip(siguientes_pasajeros, t_relat):
                yield self.env.timeout(tiempo)

                pasajero.eventos = pd.concat([
                    pasajero.eventos
                    , pd.DataFrame({
                        "pasajero": [pasajero.id]
                        , "evento": ["LLEGO A SU DESTINO"]
                        , "tiempo": [self.env.now]
                    })
                ])

                print(f"{self.env.now}: pasajero con id {pasajero.id} llego a su destino: colonia {colonia}, punto {pasajero.destino.punto}")

            print(f"{self.env.now}: vehiculo con id {vehiculo.id} volviendo al punto de taxi...")

            vehiculo.eventos = pd.concat([
                vehiculo.eventos
                , pd.DataFrame({
                    "vehiculo": [vehiculo.id]
                    , "evento": ["HACIA EL PUNTO"]
                    , "tiempo": [self.env.now]
                })
            ])
                
            yield self.env.timeout(max(t_abs))

            vehiculo.eventos = pd.concat([
                vehiculo.eventos
                , pd.DataFrame({
                    "vehiculo": [vehiculo.id]
                    , "evento": ["LLEGO AL PUNTO"]
                    , "tiempo": [self.env.now]
                })
            ])
            
            print(f"{self.env.now}: vehiculo con id {vehiculo.id} volvio al punto de taxi. Tiempo del viaje: {self.env.now - now}")

            self.punto_taxi.cola_vehiculos.append(vehiculo)


### Clase para representar vehiculos (taxis)

In [22]:
class Vehiculo:
    def __init__(self, env: Environment, id: int):
        self.env = env
        self.id = id
        self.asientos = Container(
            env
            , init = NUM_ASIENTOS_VEHICULO
            , capacity = NUM_ASIENTOS_VEHICULO
        )
        self.eventos = pd.DataFrame({
            "vehiculo": [self.id]
            , "evento": ["LLEGO AL PUNTO"]
            , "tiempo": [0]
        })

    def start(self, pasajeros: [Pasajero]):
        pass

### Clase para la simulacion

In [23]:
class Simulacion:
    def __init__(self, env: Environment):
        self.env = env
        self.colonias: [Colonia] = [ Colonia(s["nombre"], s["distancia"]) for s in COLONIAS_SETTINGS ]
        self.pasajeros = []
        self.vehiculos = [ Vehiculo(env, i) for i in range(NUM_VEHICULOS) ]
        self.punto_taxi = PuntoTaxi(env, self.colonias, self.vehiculos.copy())
        self.env.process(self.start())

    def start(self):
        while True:
            # Pasajero(s) llegan cada 1-5 minutos
            yield self.env.timeout(random.randint(1, 5))

            for _ in range(random.randint(1, 3)):
                colonia: Colonia = random.choice(self.colonias)                
                destino = Destino(colonia, random.randint(1, 10))
                
                pasajero = Pasajero(
                    self.env
                    , len(self.pasajeros) + 1
                    , destino
                    , self.punto_taxi
                )

                self.pasajeros.append(pasajero)
                self.punto_taxi.colas_pasajeros[colonia.nombre].append(pasajero)


## Iniciar simulacion

In [24]:
random.seed(333)

env = Environment()

simulacion = Simulacion(env)

env.run( until = DURACION_SIMULACION )

5: pasajero con id 1 llego al punto con destino: colonia B, punto 4
5: pasajero con id 2 llego al punto con destino: colonia B, punto 2
8: pasajero con id 3 llego al punto con destino: colonia C, punto 6
8: pasajero con id 4 llego al punto con destino: colonia B, punto 5
8: pasajero con id 5 llego al punto con destino: colonia A, punto 6
13: pasajero con id 6 llego al punto con destino: colonia B, punto 2
13: pasajeros [2, 6, 1, 4] con destino a colonia B listos para subir a taxi
13: pasajeros [2, 6, 1, 4] subieron a vehiculo con id 0
16: pasajero con id 7 llego al punto con destino: colonia B, punto 8
16: pasajero con id 8 llego al punto con destino: colonia C, punto 5
17: pasajero con id 2 llego a su destino: colonia B, punto 2
17: pasajero con id 6 llego a su destino: colonia B, punto 2
20: pasajero con id 9 llego al punto con destino: colonia B, punto 9
22: pasajero con id 1 llego a su destino: colonia B, punto 4
23: pasajero con id 10 llego al punto con destino: colonia B, punto 3

## Definir dataframe de pasajeros

In [25]:
lista_pasajeros = []
for p in simulacion.pasajeros:
    lista_pasajeros.append({
        "pasajero": p.id
        , "colonia": p.destino.colonia.nombre
        , "punto": p.destino.punto
        , "vehiculo": p.vehiculo
        , "distancia": p.destino.distancia_absoluta
    })

df_pasajeros = pd.DataFrame(lista_pasajeros)

In [26]:
df_pasajeros

,pasajero,colonia,punto,vehiculo,distancia
0,1,B,4,0.0,2.4
1,2,B,2,0.0,1.2
2,3,C,6,4.0,2.4
3,4,B,5,0.0,3.0
4,5,A,6,3.0,1.2
...,...,...,...,...,...
104,105,A,9,1.0,1.8
105,106,A,5,1.0,1.0
106,107,B,2,NaN,1.2
107,108,A,1,1.0,0.2


## Definir dataframe de eventos de pasajeros

In [27]:
lista_eventos_pasajeros = []
for p in simulacion.pasajeros:
    lista_eventos_pasajeros.append(p.eventos)

In [28]:
df_eventos_pasajeros = pd.concat(lista_eventos_pasajeros)
df_eventos_pasajeros = df_eventos_pasajeros.reset_index()
df_eventos_pasajeros.drop(["index"], axis = 1, inplace = True)

In [29]:
df_eventos_pasajeros

,pasajero,evento,tiempo
0,1,LLEGO AL PUNTO,5
1,1,SUBIO A VEHICULO,13
2,1,LLEGO A SU DESTINO,22
3,2,LLEGO AL PUNTO,5
4,2,SUBIO A VEHICULO,13
...,...,...,...
309,107,LLEGO AL PUNTO,173
310,108,LLEGO AL PUNTO,178
311,108,SUBIO A VEHICULO,178
312,108,LLEGO A SU DESTINO,178


## Definir dataframe de eventos de vehiculos

In [30]:
lista_eventos_vehiculos = []
for v in simulacion.vehiculos:
    lista_eventos_vehiculos.append(v.eventos)

In [31]:
df_eventos_vehiculos = pd.concat(lista_eventos_vehiculos)
df_eventos_vehiculos = df_eventos_vehiculos.reset_index()
df_eventos_vehiculos.drop(["index"], axis = 1, inplace = True)

In [32]:
df_eventos_vehiculos

,vehiculo,evento,tiempo
0,0,LLEGO AL PUNTO,0
1,0,HACIA EL DESTINO,13
2,0,HACIA EL PUNTO,25
3,0,LLEGO AL PUNTO,37
4,0,HACIA EL DESTINO,48
...,...,...,...
75,4,HACIA EL PUNTO,119
76,4,LLEGO AL PUNTO,130
77,4,HACIA EL DESTINO,145
78,4,HACIA EL PUNTO,159


## Preguntas a responder
- En promedio, cuanto esperaron los pasajeros en poder subirse a un vehiculo?
- Cual fue la colonia destino mas frecuente?
- En promedio, cuanto duraron los viajes de los vehiculos?
- Que vehiculo realizo la mayor cantidad de viajes?
- En promedio, que vehiculo realizo los viajes mas largos?

### 1. En promedio, cuanto esperaron los pasajeros en poder subirse a un vehiculo?

In [33]:
df_eventos_pasajeros["lag_tiempo"] = df_eventos_pasajeros.groupby("pasajero")["tiempo"].shift(1)
df1 = df_eventos_pasajeros.loc[ df_eventos_pasajeros["evento"] == "SUBIO A VEHICULO", : ].copy()
df1["tiempo_espera"] = df1["tiempo"] - df1["lag_tiempo"]

In [34]:
df1.merge(
    df_pasajeros
    , on = ["pasajero"]
    , how = "inner"
).groupby(
    "evento"
).agg({
    "tiempo_espera": ["mean"]
})

,tiempo_espera
,mean
evento,
SUBIO A VEHICULO,7.288462


### 2. Cual fue la colonia destino mas frecuente?

In [35]:
df2 = df_pasajeros.groupby(["colonia"], as_index = False).agg({
    "pasajero": ["count"]
})

df2.columns = [ "colonia", "pasajeros" ]

In [36]:
df2.sort_values("pasajeros", ascending = False)

,colonia,pasajeros
1,B,43
0,A,37
2,C,29


### 3. En promedio, cuanto duraron los viajes de los vehiculos?

In [37]:
df_eventos_vehiculos["lag_tiempo"] = df_eventos_vehiculos.groupby("vehiculo")["tiempo"].shift(3)
df3 = df_eventos_vehiculos.loc[ df_eventos_vehiculos["evento"] == "LLEGO AL PUNTO", : ].copy()
df3["tiempo_viaje"] = df3["tiempo"] - df3["lag_tiempo"]

In [38]:
df3.groupby(
    "evento"
).agg({
    "tiempo_viaje": ["mean"]
})

,tiempo_viaje
,mean
evento,
LLEGO AL PUNTO,34.166667


### 4. Que vehiculo realizo la mayor cantidad de viajes?

In [39]:
df4 = df_eventos_vehiculos.loc[ df_eventos_vehiculos["evento"] == "LLEGO AL PUNTO", : ]
df4 = df4.groupby("vehiculo").agg({ "vehiculo": ["count"] })
df4.columns = [ "viajes" ]

In [40]:
df4.sort_values("viajes", ascending = False)

,viajes
vehiculo,
0,7
4,6
3,6
2,5
1,5


### 5. En promedio, que vehiculo realizo los viajes mas largos?

In [41]:
df5 = df_pasajeros.merge(
    df_eventos_vehiculos
    , on = ["vehiculo"]
    , how = "left"
).loc[ : , ["pasajero", "vehiculo", "distancia"] ].groupby("vehiculo").agg({
    "distancia": ["mean"]
})

df5.columns = [ "distancia" ]

In [42]:
df5.sort_values("distancia", ascending = False)

,distancia
vehiculo,
2.0,3.080000
1.0,2.550000
3.0,2.100000
4.0,1.850000
0.0,1.691667
